In [172]:
### Distance calc between points

# Goal : pixel to millimeters

# We have 12 boxes
# We know where the center of every pill is


# 1. Create a grid of predefined means
# 2. Check which X and Y coordinates of the rectangle
# surround which mean
# 3. Take the colour index of the rectangle

# -- Now we know which colour is where

# 4. Compute (and plot) dist+colour from the top-left center

import cv2
import numpy as np
from imutils.object_detection import non_max_suppression
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
template_orange = cv2.imread(r"G:\\My Drive\\1. EIT Digital master\\Estland\\Semester 1\\Machine vision\\MV2021\\Homework 2\\Python OpenCV2\\Images\\Calibration\\orange_temp.png")
template_yellow = cv2.imread(r"G:\\My Drive\\1. EIT Digital master\\Estland\\Semester 1\\Machine vision\\MV2021\\Homework 2\\Python OpenCV2\\Images\\Calibration\\yellow_temp.png")
template_blue = cv2.imread(r"G:\\My Drive\\1. EIT Digital master\\Estland\\Semester 1\\Machine vision\\MV2021\\Homework 2\\Python OpenCV2\\Images\\Calibration\\blue_template.png")
template_empty = cv2.imread(r"G:\\My Drive\\1. EIT Digital master\\Estland\\Semester 1\\Machine vision\\MV2021\\Homework 2\\Python OpenCV2\\Images\\Calibration\\empty-temp.png")
img = cv2.imread(r"G:\\My Drive\\1. EIT Digital master\\Estland\\Semester 1\\Machine vision\\MV2021\\Homework 2\\Python OpenCV2\\Images\\Calibration\\checker_board6.png")

templates=[template_orange, template_yellow, template_blue, template_empty]
colours = [[0,165,255],
           [15,253,250],
           [255,0,0],
           [0,0,0]]

#template = cv2.cvtColor(template, cv2.COLOR_BGR2RGB)
(tH, tW) = templates[2].shape[:2]

probThreshold = 0.6

i = 0
points = []
mean_pills_xy = np.array([ 
                [146, 156, 0],#origin
                [266, 150, 1], 
                [386, 149, 2], #row 1, col 1
                [503, 147, 3], #row 1, col 2
                [151, 268, 4],
                [274, 264, 5],
                [390, 261, 6],
                [507, 257, 7],
                [157, 383, 8],
                [276, 375, 9],
                [395, 372, 10],
                [514, 366, 11]
                ])

for template in templates:
    bound_colour = colours[i]
    i += 1
    result = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    (yCoords, xCoords) = np.where(result >= probThreshold)
    clone = img.copy()

    # initialize our list of rectangles
    rects = []
    # loop over the starting (x, y)-coordinates again
    for (x, y) in zip(xCoords, yCoords):
        # update our list of rectangles
        rects.append((x, y, x + tW, y + tH))
    # apply non-maxima suppression to the rectangles
    pick = non_max_suppression(np.array(rects))


    # loop over the final bounding boxes
    for (startX, startY, endX, endY) in pick:
        # draw the bounding box on the image
        cv2.rectangle(img, (startX, startY), (endX, endY),
            bound_colour, 3)
        points.append((startX, startY, endX, endY, bound_colour))

    # Computing distances between points
# Origin [266, 150] (real), #origin
origin = min(points) 
x_0 = round( origin[0] + (origin[2] - origin[0])/2 )
y_0 = round( origin[1] + (origin[3] - origin[1])/2 )
origin_colour = origin[4]

center_rectangles_x = np.zeros(len(points))
center_rectangles_y = np.zeros(len(points))


for i in range(12):
    center_rectangles_x[i] = round( points[i][0] + (points[i][2] - points[i][0])/2 )
    center_rectangles_y[i] = round( points[i][1] + (points[i][3] - points[i][1])/2 )
center_rectangles = np.array([center_rectangles_x, center_rectangles_y])


# Finding the nearest mean to stick to
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(mean_pills_xy[:,0:2], mean_pills_xy[:,2])

colors = np.zeros((12,3))

for i in range(12):
    idx = neigh.predict(center_rectangles[:,i].reshape(1,-1))
    colors[idx[0]] = points[i][4]


mean_pills_xy = np.c_[mean_pills_xy, colors]

distx = np.zeros(len(points))
disty = np.zeros(len(points))
dist_man_pix = np.zeros(len(points))
dist_man_mm = np.zeros(len(points))

pix_to_mm = 106/473.31

for i in range(12):
    distx[i] = mean_pills_xy[i][0] - mean_pills_xy[0][0]
    disty[i] = mean_pills_xy[i][1] - mean_pills_xy[0][1]
    dist_man_pix[i] = distx[i] + disty[i]
    dist_man_mm[i] = round(dist_man_pix[i]*pix_to_mm)

mean_pills_xy = pd.DataFrame(np.c_[mean_pills_xy, dist_man_pix, dist_man_mm])
mean_pills_xy.columns = ['MeanX', 'MeanY', 'Label', 'ColorR', 'ColorG', 'ColorB', 'Dist2OriginPx', 'Dist2OriginMm']


cv2.imshow("Output", img)
cv2.imwrite("G:\\My Drive\\1. EIT Digital master\\Estland\\Semester 1\\Machine vision\\MV2021\\Homework 2\\Python OpenCV2\\Images\\Calibration\\pills_boundbox_calib2.png", img)
print('points:\n')
print(points)
print()

cv2.waitKey(0)

points:

[(111, 343, 203, 433, [0, 165, 255]), (230, 337, 322, 427, [0, 165, 255]), (349, 331, 441, 421, [0, 165, 255]), (467, 322, 559, 412, [0, 165, 255]), (106, 233, 198, 323, [0, 165, 255]), (344, 224, 436, 314, [0, 165, 255]), (463, 217, 555, 307, [0, 165, 255]), (218, 116, 310, 206, [0, 165, 255]), (225, 224, 317, 314, [15, 253, 250]), (456, 101, 548, 191, [15, 253, 250]), (336, 104, 428, 194, [255, 0, 0]), (93, 120, 185, 210, [0, 0, 0])]



-1

In [145]:
print(mean_pills_xy)
colors = np.zeros((12,3))

for i in range(12):
    idx = neigh.predict(center_rectangles[:,i].reshape(1,-1))
    colors[i] = points[idx[0]][4]


mean_pills_xy = np.c_[mean_pills_xy, colors]

[[146 156   0]
 [266 150   1]
 [386 149   2]
 [503 147   3]
 [151 268   4]
 [274 264   5]
 [390 261   6]
 [507 257   7]
 [157 383   8]
 [276 375   9]
 [395 372  10]
 [514 366  11]]


In [156]:
idx = neigh.predict(center_rectangles[:,0].reshape(1,-1))
print(idx)

print(center_rectangles[:,0])

[8]
[157. 388.]


[(111, 343, 203, 433, [0, 165, 255]), (230, 337, 322, 427, [0, 165, 255]), (349, 331, 441, 421, [0, 165, 255]), (467, 322, 559, 412, [0, 165, 255]), (106, 233, 198, 323, [0, 165, 255]), (344, 224, 436, 314, [0, 165, 255]), (463, 217, 555, 307, [0, 165, 255]), (218, 116, 310, 206, [0, 165, 255]), (225, 224, 317, 314, [15, 253, 250]), (456, 101, 548, 191, [15, 253, 250]), (336, 104, 428, 194, [255, 0, 0]), (93, 120, 185, 210, [0, 0, 0])]
